In [1]:
import os
import sys
import sqlite3
# Add the main module and venue_mappings modules to path so we can import them
sys.path.insert(0, os.path.abspath('../venue_mappings'))
sys.path.insert(0, os.path.abspath('../'))

from util import iterload_file_lines
from tools.utils import get_top_keywords_for_query
from venue_mappings.venue_map import VenueMapper
import networkx as nx
import matplotlib.pyplot as plt

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/lfdversluis/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/lfdversluis/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
db = sqlite3.connect('community_aminer_dblp_v11.db')
cursor = db.cursor()

cursor.execute('''CREATE TABLE IF NOT EXISTS publications
                        (id VARCHAR(64) NOT NULL,
                        venue VARCHAR(16),
                        year INTEGER,
                        volume VARCHAR(8),
                        title VARCHAR(512),
                        doi VARCHAR(128),
                        abstract TEXT,
                        citations INTEGER,
                        PRIMARY KEY (id)
                        );''')

cursor.execute('''CREATE INDEX IF NOT EXISTS ind_title
                                ON publications (title);''')

cursor.execute('''CREATE INDEX IF NOT EXISTS ind_abstract
                                ON publications (abstract);''')

cursor.execute('''CREATE INDEX IF NOT EXISTS ind_venue
                                ON publications (venue);''')

cursor.execute('''CREATE TABLE IF NOT EXISTS authors
                        (id VARCHAR(64) NOT NULL,
                        name VARCHAR(128)
                        );''')

cursor.execute('''CREATE TABLE IF NOT EXISTS author_publication
                        (author VARCHAR(64) NOT NULL,
                        publication_id VARCHAR(64) NOT NULL
                        );''')

cursor.execute('''CREATE TABLE IF NOT EXISTS collaborations
                        (author VARCHAR(64) NOT NULL,
                        co_author VARCHAR(64) NOT NULL
                        );''')

cursor.execute('''CREATE UNIQUE INDEX IF NOT EXISTS author_collaboration_pair
                        ON collaborations (author, co_author);''')

cursor.execute('''CREATE UNIQUE INDEX IF NOT EXISTS author_publication_pair
                        ON author_publication (author, publication_id);''')

db.commit()

In [3]:
def insert_article(id, venue, year, volume, title, doi, abstract, citations):
    cursor.execute("INSERT OR IGNORE INTO publications (id, venue, year, volume, title, doi, abstract, citations) VALUES(?, ?, ?, ?, ?, ?, ?, ?)",
                                (id, venue, year, volume, title, doi, abstract, citations))
    db.commit()

In [4]:
def insert_author(id, name):
    cursor.execute("INSERT OR IGNORE INTO authors (id, name) VALUES(?, ?)", (id, name))
    db.commit()

In [5]:
def insert_author_publication(author_id, publication_id):
    cursor.execute("INSERT OR IGNORE INTO author_publication (author, publication_id) VALUES(?, ?)", (author_id, publication_id))
    db.commit()

In [6]:
def insert_author_relation(author, co_author):
    cursor.execute("INSERT OR IGNORE INTO collaborations (author, co_author) VALUES(?, ?)", (author, co_author))
    db.commit()

In [7]:
# Run this cell only if you need to create or update the contents of the database.

venue_mapper = VenueMapper()
# Expects the data and format of V11 as outlined on https://aminer.org/citation
with open("/media/lfdversluis/datastore/aminer/Aminer-DBLP-author-relationship+citation-dataset/dblp_papers_v11.txt", mode="r", encoding="ISO-8859-1") as network_file:
    for article in iterload_file_lines(network_file):
        if any(a not in article for a in ["venue", "authors", "n_citation"]):
            continue  # Should never occur, but you never know...

        if "raw" not in article["venue"]:
            continue  # Sometimes there is just the field "id" in the venue object.
            
        try:
            venue = article["venue"]["raw"]
        except Exception as e:
            print(e)
            print(article)
        
        venue = venue_mapper.get_abbreviation(venue)
        if venue is None:  # We only consider papers in the system community, i.e., identified by the venue mapper.
            continue

        authors = article["authors"]
        n_citations = article["n_citation"]
        article_id = article["id"]
        article_volume = article["volume"] if "volume" in article else ""
        article_year = article["year"] if "year" in article else 0
        article_abstract = article["abstract"] if "abstract" in article else 0
        article_doi = article["doi"] if "doi" in article else 0
        article_title = article["title"] if "title" in article else 0
        
        insert_article(id=article_id, venue=venue, year=article_year, volume=article_volume, title=article_title, doi=article_doi, abstract=article_abstract, citations=n_citations)
        

        for author in authors:
            author_id = author["id"]
            author_name = author["name"] if "name" in author else ""
            
            insert_author_publication(author_id, article_id)
            
            # Insert the author
            insert_author(id=author_id, name=author_name)

            for co_author in authors:
                co_author_id = co_author["id"]
                co_author_name = co_author["name"] if "name" in co_author else ""
                
                if author_id == co_author_id:
                    continue
                
                # Put a relation of (smallest id, larger id) to only add an edge once later.
                # Doing this ensures we do not add a relationship more than once.
                if author_id < co_author_id:
                    insert_author_relation(author_id, co_author_id)
                else:
                    insert_author_relation(co_author_id, author_id)

KeyboardInterrupt: 

In [ ]:
def generate_network_for_paper_query(article_query, file_name):
    """
    This function generates a network collaboration graph based on a query provided.
    
    query: The query provided MUST select all publication ids that you want to build the network graph from.
    An example: "SELECT id FROM publications WHERE year BETWEEN 2010 AND 2019"
    file_name: The file name that should be written to.
    """
    
    query = "SELECT a.id, a.name FROM authors a INNER JOIN author_publication ap ON a.id = ap.author WHERE ap.publication_id IN ({})".format(article_query)
    query_result = cursor.execute(query)

    g = nx.Graph()
    for row in query_result:
        author_id = row[0]
        author_name = row[1]
        g.add_node(author_id, label=author_name)

    query = "SELECT DISTINCT c.author, c.co_author FROM collaborations c INNER JOIN author_publication ap ON c.author == ap.author OR c.co_author == ap.author WHERE ap.publication_id IN ({})".format(article_query)
    query_result = cursor.execute(query)
    for row in query_result:
        g.add_edge(row[0], row[1])

    nx.write_gexf(g, file_name)

In [ ]:
# Run this cell to generate the collaboration graph for "workflow scheduling" articles
query = """SELECT id 
FROM publications 
WHERE ((title like "%schedul%" or abstract like "%schedul%")
AND (title like "%workflow%" or abstract like "%workflow%"))
AND year between 2010 AND 2019"""
file_name = "collaboration-network-workflow-scheduling.gexf"
generate_network_for_paper_query(query, file_name)

In [ ]:
# Run this cell to generate the collaboration graph for "workflow formalism" articles
query = """SELECT id 
FROM publications 
WHERE ((title like "%formalism%" or abstract like "%formalism%")
AND (title like "%workflow%" or abstract like "%workflow%"))
AND year between 2010 AND 2019"""
file_name = "collaboration-network-workflow-formalism.gexf"
generate_network_for_paper_query(query, file_name)

In [ ]:
# Run this cell to generate the collaboration graph for "resource provisioning/autoscaling" articles
query = """SELECT id 
FROM publications 
WHERE ((title like "%cloud%" or abstract like "%cloud%")
AND (title like "%resource%" or abstract like "%resource%")
AND ((title like "%provision%" or abstract like "%provision%")
OR (title like "%autoscale%" or abstract like "%autoscale%")))
AND year between 2010 AND 2019"""
file_name = "collaboration-network-resource-provisioning.gexf"
generate_network_for_paper_query(query, file_name)

In [ ]:
# Run this cell to generate the collaboration graph for "cloud computing services" articles
query = """SELECT id 
FROM publications 
WHERE ((title like "%cloud%" or abstract like "%cloud%")
AND (title like "%service%" or abstract like "%service%")
AND (title like "%computing%" or abstract like "%computing%"))
AND year between 2010 AND 2019"""
file_name = "collaboration-network-cloud-computing-services.gexf"
generate_network_for_paper_query(query, file_name)